# Identify Fraud from Enron Email

## 1 Introduction

In this report, I will use machine learning to build a person of interest (poi) identifier based on Enron financial and email data made public as a result of the Enron scandal.

I will start by exploring the data set and investigating outliers. Then I will build the poi identifier following the process of feature selection/engineering, algorithm picking/tuning, validation, evaluation. 